In [1]:
import pandas as pd 
import copy

In [2]:
df_old = pd.read_excel('chen_shalev_labeld_row_data.xlsx')

In [3]:
df = df_old[:]

# time_differences

In [4]:
# Isolating relevant columns - first and last entry dates
df_entry_dates = df[['תאריך כניסה ראשונית לתוכנית', 'תאריך השמה אחרונה','תאריך דיווח השמה אחרונה']]
# all the rows with df['תאריך השמה אחרונה'] == None also df['תאריך דיווח השמה אחרונה']== None
df.loc[df['תאריך השמה אחרונה'].astype(str)=='NaT','תאריך דיווח השמה אחרונה'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [6]:
## calculating the time differnec between first to last entry date
df_entry_dates.loc[:,'days_diff'] = df['תאריך השמה אחרונה'] - df_entry_dates['תאריך כניסה ראשונית לתוכנית']

## converting "delta time" column type to integer
df.loc[:,'days_diff'] = df_entry_dates['days_diff'].dt.days

# Last match score


In [6]:
df = df.rename(columns = {'ציון התאמה אחרון':'last_match_score'})
df['last_match_score'] = round(df['last_match_score'] /10)

# unemployment_depth_months

In [7]:
df = df.rename(columns = {'עומק אבטלה בחודשים':'unemployment_depth_months'})
df['unemployment_depth_months'] = round(df['unemployment_depth_months'] /100)

# age

In [8]:
df = df.rename(columns = {'גיל':'age'})
df.loc[df['age'].between(18, 29) ,'age'] = 1829
df.loc[df['age'].between(30, 39) ,'age'] = 3039
df.loc[df['age'].between(40, 49) ,'age'] = 4049
df.loc[df['age'].between(50, 54) ,'age'] = 5054
df.loc[df['age'].between(55, 120),'age'] = 55120
df['age'] = 'age_' + df['age'].astype(str)

# education

In [9]:
df = df.rename(columns = {'רמת השכלה':'education'})
df.loc[df['education'].str.contains('יסודי חלקי'),'education'] = 'יסודי'
df.loc[df['education'].str.contains('תעודת הוראה'),'education'] = 'לימודי תעודה'
df.loc[df['education'].str.contains('הנדסאי'),'education'] = 'לימודי תעודה'
df.loc[df['education'].str.contains('טכנאי'),'education'] = 'לימודי תעודה'
df.loc[df['education'].str.contains('תעודת מקצוע'),'education'] = 'לימודי תעודה'


# languages

In [10]:
df = df.rename(columns = {'שפות':'languages'})
df['languages'] = df['languages'].str.replace('שפת אם','')\
    .str.replace('גבוהה','')\
    .str.replace('בינונית','')\
    .str.replace('בסיסית','')\
    .str.replace('-','')\
    .str.replace('-','')\
    .str.replace(' ','')

# religious

In [11]:
df = df.rename(columns = {'דת':'religious'})
df.loc[df['religious'].str.contains('נוצרי'),'religious'] = 'נוצרי'
df.loc[df['religious'].str.contains('מוסלמי'),'religious'] = 'מוסלמי'
df.loc[df['religious'].str.contains('ללא דת|נוצרי|מוסלמי|יהודי|דרוזי')==False , 'religious'] = 'other'
print(df['religious'].unique())


['מוסלמי' 'יהודי' 'נוצרי' 'other' 'דרוזי' 'ללא דת']


# programs

In [12]:
df = df.rename(columns = {'פעילויות שעבר בתכנית':'programs'})
df_programs = copy.copy(df[['programs']])

In [13]:
programs = df_programs['programs']

In [14]:

programs_list = [  'חיפוש עבודה מונחה'
                    ,'סדנת מכינה השמתית מעגלי תעסוקה'
                    ,'ייעוץ תעסוקתי'
                    ,'סדנת מכינה תהליכית מעגלי תעסוקה'
                    ,'סדנת יישומי מחשב מעגלי תעסוקה'
                    ,'שיחת אימון אישי'
                    ,'ייעוץ פסיכולוגי'
                    ,'סדנת ליווי בקבוצה קטנה'
                    ,'עברית תעסוקתית'
                    ,'זימון למנהל'
                    ,'סדנת מכינה השמתית'
                    ,'סדנת שינוי'
                    ,'סדנת מכינה מחשבים מעגלי תעסוקה'
                    ,'סדנת מכינה מעגלי תעסוקה'
                    ,'יישומי מחשב מעגלי תעסוקה'
                    ,'סדנת השמה פלוס מעגלי תעסוקה'
                    ,'סדנת מכינה תהליכית'
                    ,'סדנת מכינת השמתית מעגלי תעסוקה'
                    ,'סדנת השמה'
                    ,'מכינה תהליכית מעגלי תעסוקה'
                    ,'סדנת תהליך'
                    ,'סדנת מכינה מעורב מעגלי תעסוקה'
                    ,'סדנת השמה מעגלי תעסוקה'
                    ,'סדנת רכבת מעגלי תעסוקה'
                    ,'סדנת תהליך מעגלי תעסוקה'
                    ,'קורס יישומי מחשב'
                    ,'יישומי מחשב מעגלי תעסוקה'
                    ,'קורס יישומי מחשב'
                    ,'סדנת שינוי מעגלי תעסוקה'
                    ,'מכינה השמתית'
                 ,]


In [15]:
l =[]
for i in programs:
    p = ''
    for x in programs_list:
        if x in str(i):
            p = p + x + ','
            
    if p == '':
        p= None 
        
    l.append(p)
            

In [16]:
df_programs['programs'] = l

In [17]:
prog =  df_programs['programs'].str.get_dummies(sep=',')
prog.columns = ['programs_' + str(x).replace('.',' ') for x in prog.columns]

In [18]:
pn = prog.sum(axis=1).to_frame()
pn.columns = ['number_of_programs']

In [19]:
program_summary = pd.concat([prog,pn],axis=1)

In [20]:
l = list(df.columns)
index = l.index('programs')

In [21]:
df = df.rename(columns = {'מגדר':'gender',
                          'ארץ לידה':'origin',
                          'חד הורי':'single_parent',
                          'ילדים עד גיל 18':'childrens',
                          'אחוזי נכות':'disability',
                          'מגבלה רפואית':'medical_limitation',
                          'רשיונות נהיגה':'licenses',
                          'שירות צבאי':'military_service',
                          'תדירות_התייצבות':'frequency_stabilization',
                          '':''})

In [22]:
parameters = [ 
              'gender','age','religious','languages',
              'origin','education','military_service',
              'unemployment_depth_months','last_match_score',
              'medical_limitation','single_parent',
              'childrens','disability','licenses','days_diff',
              'label',]

In [23]:
DF = pd.concat([program_summary,df.loc[:,parameters]],axis = 1)

In [24]:
DF.head()

,programs_זימון למנהל,programs_חיפוש עבודה מונחה,programs_ייעוץ פסיכולוגי,programs_ייעוץ תעסוקתי,programs_יישומי מחשב מעגלי תעסוקה,programs_מכינה השמתית,programs_מכינה תהליכית מעגלי תעסוקה,programs_סדנת השמה,programs_סדנת יישומי מחשב מעגלי תעסוקה,programs_סדנת ליווי בקבוצה קטנה,...,military_service,unemployment_depth_months,last_match_score,medical_limitation,single_parent,childrens,disability,licenses,days_diff,label
0,0,1,0,0,0,0,0,0,0,0,...,ללא שירות,3.0,0.0,ללא מגבלות,לא,0.0,0.0,0,763.0,4
1,0,0,0,0,0,1,0,0,0,0,...,ללא שירות,2.0,0.0,ללא מגבלות,לא,4.0,0.0,0,348.0,3
2,0,0,0,1,0,0,0,0,0,0,...,ללא שירות,2.0,0.0,0,לא,0.0,0.0,B - רכב פרטי עד 3.5 טון,103.0,2
3,0,0,0,0,0,0,0,0,0,0,...,ללא שירות,2.0,0.0,בינונית,כן,0.0,0.0,0,80.0,4
4,0,0,0,0,0,0,0,0,0,0,...,ללא שירות,3.0,5.0,ללא מגבלות,לא,0.0,0.0,0,437.0,2


In [25]:
DF.to_csv('df_for_desicion_tree_xlsx.csv',index=False)

In [26]:
DF = pd.read_csv('df_for_desicion_tree_xlsx.csv')

In [27]:
DF.describe()

,programs_זימון למנהל,programs_חיפוש עבודה מונחה,programs_ייעוץ פסיכולוגי,programs_ייעוץ תעסוקתי,programs_יישומי מחשב מעגלי תעסוקה,programs_מכינה השמתית,programs_מכינה תהליכית מעגלי תעסוקה,programs_סדנת השמה,programs_סדנת יישומי מחשב מעגלי תעסוקה,programs_סדנת ליווי בקבוצה קטנה,...,programs_עברית תעסוקתית,programs_קורס יישומי מחשב,programs_שיחת אימון אישי,number_of_programs,unemployment_depth_months,last_match_score,childrens,disability,days_diff,label
count,55989.000000,55989.000000,55989.000000,55989.00000,55989.000000,55989.000000,55989.000000,55989.000000,55989.000000,55989.000000,...,55989.000000,55989.000000,55989.000000,55989.000000,55785.000000,55773.000000,55790.000000,55785.000000,39428.000000,55989.000000
mean,0.014021,0.038829,0.034989,0.15537,0.000304,0.054386,0.009430,0.000518,0.000304,0.001393,...,0.016450,0.000268,0.578114,1.204433,0.872779,3.856131,1.740133,3.796953,-273.324541,3.470735
std,0.117577,0.193189,0.183754,0.36226,0.017423,0.226779,0.096652,0.022753,0.017423,0.037299,...,0.127198,0.016366,0.493865,1.446357,0.791398,4.020191,2.053316,13.423682,1435.187507,0.890112
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8007.000000,1.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-27.000000,3.000000
50%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,4.000000,1.000000,0.000000,67.000000,4.000000
75%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,2.000000,1.000000,8.000000,3.000000,0.000000,354.000000,4.000000
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,10.000000,3.000000,10.000000,22.000000,100.000000,2034.000000,4.000000
